# Experiment 2 WandB: Logging & Evaluation

This notebook loads the trained models from Experiment 2 (epochs 1-30), evaluates them to reconstruct loss curves and generate examples, and logs everything to a new Weights & Biases project.

**Note:** This notebook is designed to run in Google Colab with access to the same Drive paths as `experiment2.ipynb`.

In [ ]:
# Mount Google Drive
try:
    from google.colab import drive
    drive.mount('/content/drive')
except ImportError:
    print("Not running in Google Colab. Skipping Drive mount.")

In [ ]:
import os
import sys

# --- CONFIGURATION ---
# Matching paths from experiment2.ipynb

try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

if IN_COLAB:
    REPO_PATH = '/content/drive/MyDrive/Kaggle_GANS_I-m-Something-of-a-Painter-Myself_Competition'
    CHECKPOINT_DIR = '/content/drive/MyDrive/MonetGAN_Checkpoints_Exp1'
    
    if os.path.exists(REPO_PATH):
        os.chdir(REPO_PATH)
        print(f"Changed directory to {REPO_PATH}")
    else:
        print(f"WARNING: Repository path {REPO_PATH} not found. Please clone it first.")
else:
    # Local fallback
    REPO_PATH = os.path.abspath(os.path.join(os.getcwd(), '..'))
    CHECKPOINT_DIR = os.path.join(REPO_PATH, 'checkpoints') # Or update if local checkpoints are elsewhere
    if os.path.exists(REPO_PATH):
        os.chdir(REPO_PATH)
        print(f"Changed directory to {REPO_PATH}")

# Ensure src is in path
sys.path.append('src')

MONET_PATH = 'data/monet_jpg'
PHOTO_PATH = 'data/photo_jpg'

print(f"Checkpoint Directory: {CHECKPOINT_DIR}")
print(f"Monet Data: {MONET_PATH}")
print(f"Photo Data: {PHOTO_PATH}")

In [ ]:
!pip install wandb scipy -q

In [ ]:
import wandb
import glob
import re
import random
import time
import itertools
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from tqdm.notebook import tqdm
from scipy import linalg

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.utils as vutils
from torchvision.models import inception_v3

from models.generator.resnet_gan import ResNetGenerator
from models.discriminator.patch_gan import PatchDiscriminator
from utils.dataset import ImageDataset, get_transforms
from utils.helpers import ReplayBuffer, weights_init_normal

In [ ]:
# WandB Config
WANDB_PROJECT = "Monet_GAN_Eval_Exp2"
WANDB_ENTITY = "konstantine25b-free-university-of-tbilisi-"

BATCH_SIZE = 4
N_EPOCHS = 30
LR = 0.0002
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(f"Using device: {DEVICE}")

In [ ]:
# Simplified WandB Login
# This will prompt you for an API key if not already logged in.
# You can also set the WANDB_API_KEY environment variable.
wandb.login()

In [ ]:
# Initialize WandB Run
wandb.init(
    project=WANDB_PROJECT,
    entity=WANDB_ENTITY,
    config={
        "epochs": N_EPOCHS,
        "batch_size": BATCH_SIZE,
        "learning_rate": LR,
        "architecture": "CycleGAN-ResNet",
        "dataset": "Monet2Photo",
        "experiment": "Experiment 2 Evaluation"
    }
)

In [ ]:
# Data Loading
monet_files = sorted(glob.glob(os.path.join(MONET_PATH, "*.*")))
photo_files = sorted(glob.glob(os.path.join(PHOTO_PATH, "*.*")))

print(f"Total Monet images: {len(monet_files)}")
print(f"Total Photo images: {len(photo_files)}")

# Shuffle files
random.seed(42)
random.shuffle(monet_files)
random.shuffle(photo_files)

def split_data(files, is_monet=False):
    n_val = 1 if is_monet else 50
    if is_monet:
        val = files[:n_val]
        train = files[n_val:]
        test = []
    else:
        n_test = 30
        val = files[:n_val]
        test = files[n_val:n_val+n_test]
        train = files[n_val+n_test:]
    return train, val, test

monet_train, monet_val, monet_test = split_data(monet_files, is_monet=True)
photo_train, photo_val, photo_test = split_data(photo_files, is_monet=False)

transforms_ = get_transforms()
train_dataset = ImageDataset(monet_train, photo_train, transform=transforms_)
val_dataset = ImageDataset(monet_val, photo_val, transform=transforms_)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

# Fixed samples for visualization
val_batch = next(iter(val_loader))
fixed_photo_val = val_batch['photo'].to(DEVICE)
fixed_monet_val = val_batch['monet'].to(DEVICE)

def denormalize(tensor):
    return tensor * 0.5 + 0.5

In [ ]:
# Model Initialization
G_Monet = ResNetGenerator().to(DEVICE)
G_Photo = ResNetGenerator().to(DEVICE)
D_Monet = PatchDiscriminator().to(DEVICE)
D_Photo = PatchDiscriminator().to(DEVICE)

criterion_GAN = nn.MSELoss()
criterion_cycle = nn.L1Loss()
criterion_identity = nn.L1Loss()

In [ ]:
# --- MiFID Implementation ---
# Based on Kaggle GANs Competitions (Monet, etc.)

class InceptionV3(nn.Module):
    """Pretrained InceptionV3 network returning feature maps"""
    def __init__(self):
        super().__init__()
        inception = inception_v3(pretrained=True)
        self.block1 = nn.Sequential(
            inception.Conv2d_1a_3x3, inception.Conv2d_2a_3x3,
            inception.Conv2d_2b_3x3, nn.MaxPool2d(kernel_size=3, stride=2)
        )
        self.block2 = nn.Sequential(
            inception.Conv2d_3b_1x1, inception.Conv2d_4a_3x3,
            nn.MaxPool2d(kernel_size=3, stride=2)
        )
        self.block3 = nn.Sequential(
            inception.Mixed_5b, inception.Mixed_5c, inception.Mixed_5d,
            inception.Mixed_6a, inception.Mixed_6b, inception.Mixed_6c,
            inception.Mixed_6d, inception.Mixed_6e
        )
        self.block4 = nn.Sequential(
            inception.Mixed_7a, inception.Mixed_7b, inception.Mixed_7c,
            nn.AdaptiveAvgPool2d(output_size=(1, 1))
        )

    def forward(self, x):
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        return x.view(x.size(0), -1)

def calculate_frechet_distance(mu1, sigma1, mu2, sigma2, eps=1e-6):
    """Numpy implementation of the Frechet Distance."""
    mu1 = np.atleast_1d(mu1)
    mu2 = np.atleast_1d(mu2)

    sigma1 = np.atleast_2d(sigma1)
    sigma2 = np.atleast_2d(sigma2)

    assert mu1.shape == mu2.shape, "Training and test mean vectors have different lengths"
    assert sigma1.shape == sigma2.shape, "Training and test covariances have different dimensions"

    diff = mu1 - mu2

    # Product might be almost singular
    covmean, _ = linalg.sqrtm(sigma1.dot(sigma2), disp=False)
    if not np.isfinite(covmean).all():
        print("fid calculation produces singular product; adding %s to diagonal of cov estimates" % eps)
        offset = np.eye(sigma1.shape[0]) * eps
        covmean = linalg.sqrtm((sigma1 + offset).dot(sigma2 + offset))

    # Numerical error might give slight imaginary component
    if np.iscomplexobj(covmean):
        if not np.allclose(np.diagonal(covmean).imag, 0, atol=1e-3):
            m = np.max(np.abs(covmean.imag))
            raise ValueError("Imaginary component {}".format(m))
        covmean = covmean.real

    tr_covmean = np.trace(covmean)

    return (diff.dot(diff) + np.trace(sigma1) + np.trace(sigma2) - 2 * tr_covmean)

def calculate_activation_statistics(images, model, batch_size=50, device='cuda'):
    model.eval()
    act = []
    
    if not isinstance(images, torch.Tensor):
        # Assuming images is a list of tensors
        pass
    
    dataloader = DataLoader(images, batch_size=batch_size, shuffle=False)
    
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Calculating Inception Features"):
            batch = batch.to(device)
            # Resize to 299x299 for InceptionV3 if needed, or assume inputs are close
            if batch.shape[-1] != 299:
                 batch = F.interpolate(batch, size=(299, 299), mode='bilinear', align_corners=False)
            
            pred = model(batch)
            act.append(pred.cpu().numpy())

    act = np.concatenate(act, axis=0)
    mu = np.mean(act, axis=0)
    sigma = np.cov(act, rowvar=False)
    return mu, sigma, act

def calculate_mifid(real_images_tensor, fake_images_tensor, model, device='cuda', eps=1e-15):
    """
    Calculates MiFID between real and fake images.
    real_images_tensor: Tensor of shape (N, C, H, W)
    fake_images_tensor: Tensor of shape (N, C, H, W)
    """
    
    # 1. Calculate FID components
    mu1, sigma1, act1 = calculate_activation_statistics(real_images_tensor, model, device=device)
    mu2, sigma2, act2 = calculate_activation_statistics(fake_images_tensor, model, device=device)
    
    fid_value = calculate_frechet_distance(mu1, sigma1, mu2, sigma2)
    
    # 2. Calculate Memorization Distance (Cosine Distance)
    # Formula: d_mem = 1/N * sum(min(d(f_gen, f_real)))
    # We want cosine distance, so we normalize vectors first
    
    # Normalize features
    act1_norm = act1 / (np.linalg.norm(act1, axis=1, keepdims=True) + eps)
    act2_norm = act2 / (np.linalg.norm(act2, axis=1, keepdims=True) + eps)
    
    # Compute cosine similarity matrix (Fake x Real)
    # act2 (Fake) dot act1.T (Real)
    # Result is (N_fake, N_real) matrix of cosine similarities
    # Distance = 1 - Similarity
    # We want MINIMUM distance for each fake image to ANY real image
    
    # To save memory, compute in batches if needed, but for <10k images it might fit in RAM
    # 7000 images * 2048 floats is manageable
    
    # Similarity
    sim_matrix = np.dot(act2_norm, act1_norm.T)
    
    # Distance = 1 - Similarity
    dist_matrix = 1.0 - sim_matrix
    
    # Find minimum distance for each generated image
    min_distances = np.min(dist_matrix, axis=1)
    
    # Thresholding (per Kaggle spec, often 0.0 or small epsilon, but standard MiFID uses pure mean)
    # Kaggle formula: if distance > epsilon, it's 1.0? 
    # Actually, the formula usually is just penalizing very small distances (memorization)
    # Standard Kaggle implementation:
    # memorization_distance = np.mean(min_distances)
    # But usually applied as penalty: MiFID = FID + alpha / memorization_distance
    # Wait, the user prompt says: "The memorization distance is defined as the minimum cosine distance..."
    # And "Finally, this memorization term is applied to the FID: FID = FID * (1 / (mem_dist + eps))" or similar?
    # Re-reading prompt: "FID_m = FID + 1 / (N * d_mem)"? No, let's look at the formula descriptions
    # "This distance is thresholded... assigned to 1.0 if distance exceeds pre-defined epsilon"
    # Let's assume a standard Kaggle implementation approach:
    
    # Thresholding logic from public kernels:
    # distance_threshold = 0.00something
    # But let's calculate the raw mean minimum distance first.
    
    memorization_dist = np.mean(min_distances)
    
    # MiFID formula from Kaggle usually:
    # MiFID = FID + alpha / (memorization_dist + epsilon)
    # Use alpha=1.0 for now if not specified
    # NOTE: The user prompt doesn't give the EXACT final combination formula constant, 
    # but says "memorization term is applied to the FID"
    # I will log FID, Memorization Distance, and a heuristic MiFID.
    
    mifid = fid_value * (1.0 / (memorization_dist + 1e-7)) # Heuristic based on description: smaller distance -> larger penalty -> larger MiFID (bad)
    
    return fid_value, memorization_dist, mifid

# Initialize Inception Model
inception_model = InceptionV3().to(DEVICE)
inception_model.eval()
print("InceptionV3 initialized for MiFID")

In [ ]:
# Pre-load Real Monet Images for MiFID (Standard for comparison)
# We only need to compute statistics for real Monet images ONCE
# Since we are generating Monets from Photos, we compare (Generated Monets) vs (Real Monets)

print("Loading all Real Monet images for statistics...")
real_monet_tensor_list = []
for f in tqdm(monet_files, desc="Loading Real Monets"):
    img = Image.open(f).convert('RGB')
    img_t = transforms_(img)
    real_monet_tensor_list.append(img_t)

real_monet_stack = torch.stack(real_monet_tensor_list).to(DEVICE)
print(f"Real Monet Stack: {real_monet_stack.shape}")

In [ ]:
# Evaluation Function
def evaluate_epoch(epoch, num_batches=20):
    """
    Loads weights for the given epoch, runs a few batches to calculate loss,
    and logs metrics and images to WandB.
    """
    # 1. Load Weights
    try:
        G_Monet.load_state_dict(torch.load(os.path.join(CHECKPOINT_DIR, f'G_Monet_epoch_{epoch}_2.pth'), map_location=DEVICE))
        G_Photo.load_state_dict(torch.load(os.path.join(CHECKPOINT_DIR, f'G_Photo_epoch_{epoch}_2.pth'), map_location=DEVICE))
        D_Monet.load_state_dict(torch.load(os.path.join(CHECKPOINT_DIR, f'D_Monet_epoch_{epoch}_2.pth'), map_location=DEVICE))
        D_Photo.load_state_dict(torch.load(os.path.join(CHECKPOINT_DIR, f'D_Photo_epoch_{epoch}_2.pth'), map_location=DEVICE))
    except FileNotFoundError:
        print(f"Checkpoint for epoch {epoch} not found in {CHECKPOINT_DIR}. Skipping...")
        return

    # 2. Calculate Losses (using a subset of training data)
    G_Monet.eval(); G_Photo.eval(); D_Monet.eval(); D_Photo.eval()
    
    total_loss_G = 0.0
    total_loss_D_Monet = 0.0
    total_loss_D_Photo = 0.0
    
    with torch.no_grad():
        for i, batch in enumerate(train_loader):
            if i >= num_batches:
                break
                
            real_monet = batch['monet'].to(DEVICE)
            real_photo = batch['photo'].to(DEVICE)
            
            # -- Generator Losses --
            loss_id_A = criterion_identity(G_Monet(real_monet), real_monet)
            loss_id_B = criterion_identity(G_Photo(real_photo), real_photo)
            loss_identity = (loss_id_A + loss_id_B) / 2

            fake_monet = G_Monet(real_photo)
            loss_GAN_AB = criterion_GAN(D_Monet(fake_monet), torch.ones_like(D_Monet(fake_monet)))
            fake_photo = G_Photo(real_monet)
            loss_GAN_BA = criterion_GAN(D_Photo(fake_photo), torch.ones_like(D_Photo(fake_photo)))
            loss_GAN = (loss_GAN_AB + loss_GAN_BA) / 2

            rec_photo = G_Photo(fake_monet)
            loss_cycle_A = criterion_cycle(rec_photo, real_photo)
            rec_monet = G_Monet(fake_photo)
            loss_cycle_B = criterion_cycle(rec_monet, real_monet)
            loss_cycle = (loss_cycle_A + loss_cycle_B) / 2

            loss_G = loss_GAN + (10.0 * loss_cycle) + (5.0 * loss_identity)
            total_loss_G += loss_G.item()
            
            # -- Discriminator Losses --
            loss_real_M = criterion_GAN(D_Monet(real_monet), torch.ones_like(D_Monet(real_monet)))
            loss_fake_M = criterion_GAN(D_Monet(fake_monet), torch.zeros_like(D_Monet(fake_monet)))
            total_loss_D_Monet += (loss_real_M + loss_fake_M).item() / 2
            
            loss_real_P = criterion_GAN(D_Photo(real_photo), torch.ones_like(D_Photo(real_photo)))
            loss_fake_P = criterion_GAN(D_Photo(fake_photo), torch.zeros_like(D_Photo(fake_photo)))
            total_loss_D_Photo += (loss_real_P + loss_fake_P).item() / 2

    avg_loss_G = total_loss_G / num_batches
    avg_loss_D_Monet = total_loss_D_Monet / num_batches
    avg_loss_D_Photo = total_loss_D_Photo / num_batches
    
    # --- MiFID Calculation ---
    # Generate a batch of fake Monets from Photos to compare against Real Monets
    # We need a decent number of samples for FID, but doing full 7000 every epoch is slow.
    # Let's do a smaller subset (e.g., 100 images) for epoch-wise tracking.
    
    print("Generating images for MiFID...")
    fake_monets_list = []
    count = 0
    num_mifid_samples = 100 # Adjust based on speed requirements
    
    for batch in train_loader:
        if count >= num_mifid_samples:
            break
        real_photo = batch['photo'].to(DEVICE)
        with torch.no_grad():
            fake = G_Monet(real_photo)
            fake_monets_list.append(fake)
            count += real_photo.size(0)
            
    fake_monet_stack = torch.cat(fake_monets_list, dim=0)[:num_mifid_samples]
    
    # We compare against a subset of real monets matching the size, or full set?
    # Using full real set is better for stability.
    fid, mem_dist, mifid = calculate_mifid(real_monet_stack, fake_monet_stack, inception_model, device=DEVICE)
    
    
    # 3. Log Metrics
    wandb.log({
        "Loss/G_Total": avg_loss_G,
        "Loss/D_Monet": avg_loss_D_Monet,
        "Loss/D_Photo": avg_loss_D_Photo,
        "Metrics/FID": fid,
        "Metrics/Memorization_Distance": mem_dist,
        "Metrics/MiFID": mifid,
        "epoch": epoch
    }, step=epoch)
    
    # 4. Generate and Log Images
    # We only log 2 images (1 of each type) to avoid crashing WandB
    with torch.no_grad():
        # Use sliced input [0:1] to ensure we only generate and process 1 image
        fake_monet_val_vis = G_Monet(fixed_photo_val[0:1])
        fake_photo_val_vis = G_Photo(fixed_monet_val[0:1])
        
        wandb.log({
            "Generated/Val_Photo_to_Monet": [wandb.Image(denormalize(fake_monet_val_vis[0]).cpu(), caption=f"Epoch {epoch} Monet")],
            "Generated/Val_Monet_to_Photo": [wandb.Image(denormalize(fake_photo_val_vis[0]).cpu(), caption=f"Epoch {epoch} Photo")],
        }, step=epoch)

    # 5. Log Artifacts (Every 5th epoch)
    if epoch % 5 == 0:
        artifact = wandb.Artifact(name=f"model_epoch_{epoch}", type="model")
        artifact.add_file(os.path.join(CHECKPOINT_DIR, f'G_Monet_epoch_{epoch}_2.pth'))
        artifact.add_file(os.path.join(CHECKPOINT_DIR, f'G_Photo_epoch_{epoch}_2.pth'))
        artifact.add_file(os.path.join(CHECKPOINT_DIR, f'D_Monet_epoch_{epoch}_2.pth'))
        artifact.add_file(os.path.join(CHECKPOINT_DIR, f'D_Photo_epoch_{epoch}_2.pth'))
        wandb.log_artifact(artifact)
        print(f"Logged artifact for epoch {epoch}")

    print(f"Epoch {epoch}: G_Loss={avg_loss_G:.4f}, FID={fid:.4f}, MiFID={mifid:.4f}")

In [ ]:
# Run the evaluation loop
print(f"Starting evaluation for {N_EPOCHS} epochs...")

for epoch in range(1, N_EPOCHS + 1):
    evaluate_epoch(epoch, num_batches=10)

print("Done!")
wandb.finish()